<a href="https://colab.research.google.com/github/Buzon-coder/QRT-Electricity-price/blob/main/QRT_%C3%A9lec_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importation de mes bibliothèques

In [68]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, pearsonr
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler

# Chargement des fichiers csv

In [62]:
X_test = pd.read_csv("/content/drive/MyDrive/Colab data/X_test_final.csv")
X_train = pd.read_csv("/content/drive/MyDrive/Colab data/X_train_NHkHMNU.csv")
y_train = pd.read_csv("/content/drive/MyDrive/Colab data/y_train_ZAN5mwg.csv")

# Encodage de ma (seule) variable catégorielle COUNTRY

In [63]:
X_train_encode = X_train.copy()
X_train_encode["COUNTRY_FR"] = (X_train_encode["COUNTRY"] == "FR").astype(int)
X_train_encode["COUNTRY_DE"] = (X_train_encode["COUNTRY"] == "DE").astype(int)
X_train_encode = X_train_encode.drop(columns = ["COUNTRY"])

# Calcul des corrélations de spearman entre mes features et mes valeurs cibles

In [81]:
# Je vérifie que X_train_encode et y_train ont les mêmes index
print((X_train_encode.index == y_train.index).all())
# c'est le cas, donc je peux utiliser la fonction corrwith pour calculer la corrélation entre les features de X_train_encode et y_train
corrs = X_train_encode.corrwith(y_train["TARGET"], method = 'spearman')
corrs_abs = corrs.abs().sort_values(ascending=False)
# print(corrs_abs.head(20))
# print(corrs_abs.tail(18))

True


J'obtiens que les features les plus corrélées (en valeur absolue) de X_train et avec les TARGET (de y_train) sont par ordre décroissant : DE_NET_IMPORT, DE_NET_EXPORT, DE_WINDPOW, DE_RESIDUAL_LOAD, FR_WINDPOW,DE_HYDRO, DE_GAS, CARBON_RET, DE_WIND, DE_COAL, GAS_RET, FR_HYDRO,FR_WIND, DE_CONSUMPTION, FR_RAIN, FR_DE_EXCHANGE, DE_FR_EXCHANGE, FR_COAL, FR_TEMP, DE_LIGNITE.
\

De même, j'obtiens les features les moins corrélées (en valeur absolue) de X_train avec les TARGET sont (de la plus corrélées à la moins corrélée) : DE_RAIN, DE_SOLAR, FR_SOLAR, FR_CONSUMPTION, ID, COUNTRY_FR, COUNTRY_DE, DE_TEMP, FR_RESIDUAL_LOAD, COAL_RET, FR_GAS, FR_NET_EXPORT, FR_NET_IMPORT, DE_NUCLEAR, DAY_ID, FR_NUCLEAR.
\

Regarder les corrélations permet de comprendre quelles features influencent TARGET, et permet d'interpréter phyisquement le modèle. Ces features représentent les effets dominants.

# Preprocessing des features

# a) Détection des NaN

In [65]:
# print(X_train_encode.isna().sum())
# print(X_test.isna().sum())

Mes 12 colonnes qui comportent des NaN sont les mêmes dans X_test et X_train, et ce sont :
- toutes mes variables météorologiques
- mes variables de mesure d'utilisation électrique journalière (x_NET_IMPORT,x_NET_EXPORT, x_y_EXCHANGE).

 # b) Imputation des NaN (on obtient X_imputed)

Je vais faire une imputation par régression (IterativeImputer()), c'est-à-dire entraîner un petit modèle pour chaque variable manquante (x_TEMP) à partir des autres features corrélées (x_RAIN, x_WIND).
\

L'idée de IterativeImputer() pour imputer les NaN de la colonne X_j, c'est d'utiliser toutes les autres colonnes (tous les X_k avec k != j) comme features explicatives, puis d'entraîner un modèle de régression sur les lignes où X_j n'est pas manquant.
\

Remarque : Il faudra faire attention, car j'ai rempli des lignes correspondant à DE dans des colonnes sur des données FR, et réciproquement.

In [66]:
imputer = IterativeImputer()
X_imputed = imputer.fit_transform(X_train_encode)
X_imputed = pd.DataFrame(X_imputed, columns=X_train_encode.columns, index=X_train_encode.index) # la fonction IterativeImputer renvoie un numpy.ndarray
                                                                                                # donc je le retransforme en DataFrame ensuite

# print(type(X_imputed))
# print(X_imputed.isna().sum()) # --> toutes mes valeurs NaN ont bien été imputées

# c) Vérification de la multicolinéarité entre les features

La VIF (variance inflation factor, facteur d'inflation de la variance) d'une variable X_i mesure à quel point elle peut être expliquée par les autres variables du dataset.

In [67]:
vif = pd.DataFrame()
vif["feature"] = X_imputed.columns
vif["VIF"] = [variance_inflation_factor(X_imputed.values, i) for i in range(X_imputed.shape[1])]

high_vif_features = vif.loc[vif["VIF"] >=10, "feature"].tolist()
print(high_vif_features)
print(len(high_vif_features))

['ID', 'DE_CONSUMPTION', 'FR_CONSUMPTION', 'DE_FR_EXCHANGE', 'FR_DE_EXCHANGE', 'DE_NET_EXPORT', 'FR_NET_EXPORT', 'DE_NET_IMPORT', 'FR_NET_IMPORT', 'FR_GAS', 'FR_NUCLEAR', 'DE_SOLAR', 'FR_SOLAR', 'DE_WINDPOW', 'FR_WINDPOW', 'DE_RESIDUAL_LOAD', 'FR_RESIDUAL_LOAD', 'COUNTRY_FR']
18


J'ai 18 features qui ont un facteur d'inflation de la variance >= 10, donc j'ai beaucoup de multicolinéarité, d'où l'intérêt d'utiliser un algorithme avec pénalisation.

\

Plus tard, je vais déterminer quelles sont les features les plus utiles avec Ridge et Lasso.

# d) Standardistation des features (on obtient X_scaled)

In [77]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)
X_scaled = pd.DataFrame(X_scaled, columns = X_imputed.columns, index = X_imputed.index)
# print(X_scaled["FR_GAS"].describe())
# print(X_scaled["FR_DE_EXCHANGE"].describe())

En fait à refaire pour faire gaffe aux lignes DE dans les colonnes FR.
prompt ChatGPT :

J'ai fait ça : imputer = IterativeImputer() X_imputed = imputer.fit_transform(X_train_encode) X_imputed = pd.DataFrame(X_imputed, columns=X_train_encode.columns, index=X_train_encode.index) Mais en fait, je veux que deux lignes d'une même colonne qui ont le même DAY_ID aient la même valeur, et que cette valeur soit choisie en fonction de la colonne. S'il s'agit de la colonne FR_RAIN, je veux que la valeur du NaN aux lignes COUNTRY_DE == 1 soit la même que la valeur obtenue par IterativeImputer() à la ligne COUNTRY_FR == 1 pour un même DAY_ID. Donc en fait, ce serait peut-être bien de faire un dataset pour les features françaises (FR_RAIN, FR_WIND, FR_TEMP, FR_NET_IMPORT, FR_NET_EXPORT) dans lequel on ne met que les lignes qui ont COUNTRY_FR == 1 et de faire le IterativeImputer() sur ce dataset uniquement pour ces features, et de même de faire un dataset pour les features allemandes (DE_RAIN, DE_WIND, DE_TEMP, DE_NET_IMPORT, DE_NET_EXPORT) et de faire le IterativeImputer() sur ce dataset uniquement pour ces features, qu'en penses-tu ? Ensuite, il me manquera les colonnes DE_FR_EXCHANGE et FR_DE_EXCHANGE à imputer, et comme les valeurs de DE_FR_EXCHANGE sont exactement l'opposée des valeurs de FR_DE_EXCHANGE, je n'aurai besoin que de déduire les valeurs à imputer aux NaN d'une colonne pour déduire celles à imputer à l'autre. Enfin, j'aimerais réunir mes deux datasets (un par pays) dont j'ai imputé les NaN en un seul. Est-ce que tu vois un problème dans mon raisonnement ?



Mais pour imputer des valeurs NaN dans ma colonne FR_NET_IMPORT par exemple, je veux utiliser toutes les colonnes de mon dataset de départ, pas uniquement les colonnes fr_cols. Car elles vont toutes avoir une influence directe ou indirecte sur la valeur à imputer. Par contre, je ne veux me baser que sur les lignes qui vérifient COUNTRY_FR == 1. J'ai l'impression que dans ton code tu as enlevé les colonnes autres que fr_cols pour mon dataset français, et les colonnes autres que de_cols pour mon dataset allemand, est-ce que je me trompe? Moi je ne veux pas enlever ces colonnes, ce que je veux, c'est utiliser les lignes adéquat, donc faire un groupby country pour obtenir chaque dataset.



regarder chat ML 2.4.2 (QR élec), normalement il m'a fait le code

Mes features sont bien centrées en 0 (moyenne = 0) et d'écart-type égal à 1.

# e) Feature Engineering

# f) Réduction de dimension (optionnel)